# Aim

Given a large set of sequences or graphs with ordered vertices find small vertex ordered subsequences that are most discriminative for the set.

Steps:
- devise a negative set
- learn a discriminative model
- annotate importance on vertices
- extract max subarrays 
- cluster them 
 - use fast EDeN string kernel 
 - custering algorithm
 
Output: 
1. all sequence motives in each cluster
2. all initial sequences with motif location (begin,end) and cluster id (build regex from all seqs in cluster and run a find iterator)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#code for making artificial dataset
import random
def random_string(length,alphabet_list):
    rand_str = ''.join(random.choice(alphabet_list) for i in range(length))
    return rand_str

def perturb(seed,alphabet_list,p=0.5):
    seq=''
    for c in seed:
        if random.random() < p: c = random.choice(alphabet_list)
        seq += c
    return seq

def make_artificial_dataset(alphabet='ACGU', motives=None, motif_length=6, sequence_length=100, n_sequences=1000, n_motives=2, p=0.2):
    alphabet_list=[c for c in alphabet]
    
    if motives is None:
        motives=[]
        for i in range(n_motives):
            motives.append(random_string(motif_length,alphabet_list))
    else:
        motif_length = len(motives[0])
        n_motives = len(motives)
        
    flanking_length = (sequence_length - motif_length ) / 2
    n_seq_per_motif = n_sequences / n_motives

    counter=0
    seqs=[]
    for i in range(n_seq_per_motif):
        for j in range(n_motives):
            left_flanking = random_string(flanking_length,alphabet_list)
            right_flanking = random_string(flanking_length,alphabet_list)
            noisy_motif = perturb(motives[j],alphabet_list,p)
            seq = left_flanking + noisy_motif + right_flanking
            seqs.append(('>ID%d'%counter,seq))
            counter += 1
    return motives, seqs

In [3]:
from eden.motif import SequenceMotif
help(SequenceMotif)

Help on class SequenceMotif in module eden.motif:

class SequenceMotif(__builtin__.object)
 |  Methods defined here:
 |  
 |  __init__(self, min_subarray_size=7, max_subarray_size=10, min_motif_count=1, min_cluster_size=1, training_size=None, negative_ratio=2, shuffle_order=2, n_iter_search=1, complexity=4, nbits=20, clustering_algorithm=None, n_jobs=4, n_blocks=8, block_size=None, pre_processor_n_jobs=4, pre_processor_n_blocks=8, pre_processor_block_size=None, random_state=1)
 |  
 |  fit(self, seqs, neg_seqs=None)
 |      Builds a discriminative estimator.
 |      Identifies the maximal subarrays in the data.
 |      Clusters them with the clustering algorithm provided in the initialization phase.
 |      For each cluster builds a fast sequence search model (Aho Corasick data structure).
 |  
 |  fit_predict(self, seqs, return_list=False)
 |  
 |  fit_transform(self, seqs, return_match=False)
 |  
 |  info(self)
 |  
 |  load(self, obj)
 |  
 |  predict(self, seqs, return_list=False)

#Experimental Setup

In [4]:
#setup parameters
alphabet='ACGU'
motives=['AAAAAAAAAA','CCCCCCCCCC','GGGGGGGGGG','UUUUUUUUUU']
sequence_length=100
n_sequences=100
p=0.3

#make dataset
motives, seqs = make_artificial_dataset(alphabet=alphabet,motives=motives,sequence_length=sequence_length,n_sequences=n_sequences,p=p)

#display
print 'Motives and sample of their perturbed variants:'
alphabet_list=[c for c in alphabet]
for motif in motives: 
    print
    print motif,
    for i in range(9):
        print perturb(motif,alphabet_list,p=p),

Motives and sample of their perturbed variants:

AAAAAAAAAA ACUAUAAAAA AAAAAAAAAA AAAAAGAAAU AGAAAUAAUA AAAAAAAAAC AAACAAAACA AAAAAAAAAC AACAAAGAUA AAAGAAACAA
CCCCCCCCCC CCCCCCCCCC CCCCCCACCC CCCCCCCCGC CCCCCUCUCU AGCCCCCCUC CCUCCCCCCU CCCCGACGCC CCCCCCCCAU CCCCCCCCCA
GGGGGGGGGG GGGGGGAGGG GAGGGGGGGG UGGGGGUAGG GGGGAGGGGG GGGGGGGGUG UGCGGUCGGC GGGGUUAGGG GCGGGGGGGC GGGGGGGGGG
UUUUUUUUUU UUUUUUGUUU UUCUUAGUAU UUUGUUUUCU UUUCUUUAUC UACUUUUCUU UGCUUGGCUG UCUUUUUUAU AUUGUCUUCU UUUUUAUUUU


In [5]:
#save to file
fname='artificial_motif_search_dataset.fa'
with open(fname,'w') as f:
    for header,seq in seqs: 
        f.write(header+"\n")
        f.write(seq+"\n")

#save explicit negative sequences
from eden.modifier.seq import seq_to_seq, shuffle_modifier
neg_seqs = list(seq_to_seq(seqs, modifier=shuffle_modifier, times=2, order=2))
fname='artificial_motif_search_dataset_negatives.fa'
with open(fname,'w') as f:
    for header,seq in neg_seqs: 
        f.write(header+"\n")
        f.write(seq+"\n")

In [6]:
print seqs

[('>ID0', 'UACCGAUCCAGUAGCUUAUGAAAACACGGGAGAGGUACGCUGCGGAAAAAGAAAAUAGGUCCCCCUGAGAGGUUAAUAGGGUUCUGACACCCUUCUGGAC'), ('>ID1', 'AAGAGUUAGACUCAGGCCCGUAGUAUUAGCAUAGAGGCAGCGCGCCCCCCCGCCACAGGGUAAAGUCCUUACAGGCCCGGGUACGGAGGUGGCCCAGUGC'), ('>ID2', 'CUCGUAGCUGUUAAUGGUAUAGGCUUUAUGUGCCCUAGCAAAUAGUCCUGCUGGGCAAUUACUUACUCGCACGCGCGGCCUCGGCUUUCUGCAUGAGCAA'), ('>ID3', 'CGAAAAAAGUGUCUUUCAGAACGGGUGCCGCUUCAUUUUCAGACAUUUUUGUUUCGAACAGACCUUAGUCCGGGACAGGAAGCAUACAGGUUCAUGGUAU'), ('>ID4', 'CGGUUCCAUCGGGACUUCGUUGCUGGACCUUAGAUCCAGUUGGGAAAAAAAACAAUCGAGUGGUGACACCGAACUUCCUGUUUCUAUGCAGAUCCGGUUC'), ('>ID5', 'GAAUCGUCCUGAGUAAACGUACUGAGCCUCUGUGAAUUUCCUAUGCCCCCCCCACUGGCGACCUUUCAAUUAUCUCGGAUGAGGUCCGAUCAGGACUCUG'), ('>ID6', 'GACUUACGCAUCGGAUCCUGUUUAUAAAGGUGUAUCGAUACUUCCGGGAGCAGGCAUCGGUCGCCCUAGACCCUCUAACUCUAGCUUCUUUAAGAUUAAU'), ('>ID7', 'UAUACGGCCUUUAGCUUUAACAAGCCGUUCGAUUCGGAACUCGGCAUUUUUUUUUACUAGGUCUGGCAAAGGAUAGACUACGUUAUACGUGGGAUAAUAG'), ('>ID8', 'UGCCUGCGUUAGGACAUCACUUGAUACCCCUUUUCAUGAAAUCGAAAAAAACAAAAUGAGUCUCCCACGAGUACGGA

In [7]:
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=2)

In [8]:
%%time
from sklearn.cluster import Birch
ca = Birch(threshold=0.1, n_clusters=4, branching_factor=50)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(training_size=100, complexity=2, nbits=14, clustering_algorithm=ca)
seqmot.fit(seqs, neg_seqs)
seqmot.save('seqmot')

Positive data: Instances: 100 ; Features: 16385 with an avg of 454 features per instance
Negative data: Instances: 200 ; Features: 16385 with an avg of 458 features per instance
Elapsed time: 3.4 secs
model induction: 100 positive instances 3 s
motives extraction: 63 motives in 0s
motives clustering: 4 clusters in 0s
after filtering: 57 motives 4 clusters in 0s
motif model construction in 0s
updated motif counts in 0s
CPU times: user 2.65 s, sys: 448 ms, total: 3.1 s
Wall time: 4.88 s


In [9]:
for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

0
CCCCCCCCA 4
CCCCCCCCCU 3
CCCCCCCCCC 3
UCCCCCCCCC 2
CUCCCCCCCG 2
CACCCCCCC 2
UCUCUCUCCG 1
UCUCGGCCGU 1
GCCCCCCCG 1
GCCCCCCACC 1
CCUCUGUGA 1
CCUCACCACA 1
CCCCUGGGC 1
CCCCCUCUCC 1
CCCCCGCCCC 1
CCCCCCCACC 1
ACCCCCCCA 1
ACCCACCCA 1
1
AGGGGGGGGG 2
UGGGGUUAU 1
GUGGGGGGG 1
GGUGGUGGUG 1
GGGGUGGGGA 1
GGGGGGGGGG 1
GGGGGGGGGA 1
CGUGGGGGGC 1
CGGGUGCCG 1
CGGGUGAGA 1
CGGGGCGGU 1
CCACGGGGGA 1
AUUGGGGGAG 1
AUCCGGGGA 1
AGGGGCUCC 1
ACGUGGUGC 1
AAAGGGGUGA 1
2
UAAAAAAAA 5
AAAAAGAAAA 3
AAAAAAAAA 3
AAAAAAGAGA 2
AAAAAAAGAA 2
AAAAAAAAG 2
AAAAAAAAAA 2
GAAAAAAAA 1
AAGAAAAAAG 1
AAAAUAAAAA 1
AAAAAAGUG 1
3
UUUUUUUUU 4
UUUUUUUUUU 2
UUUUUUUUCU 2
UUUUUGUUU 1
UUUAUUUUUA 1
UCUUUUUAUU 1
UCUUUAUUUU 1
UAUUUUUUUU 1
UAUUUUUAUU 1
GUUUAUAAAG 1
AUUUUUUAU 1


In [10]:
from eden.motif import SequenceMotif
seqmot2 = SequenceMotif()
seqmot2.load('seqmot')

predictions=seqmot2.predict(seqs, return_list=True)
for p in predictions: print p

[2]
[0]
[]
[2, 1, 3]
[2]
[0]
[3]
[3]
[]
[0]
[1]
[]
[]
[]
[1]
[1]
[]
[0]
[]
[3]
[2]
[0]
[]
[3]
[]
[0]
[]
[3]
[]
[]
[1]
[1, 0, 3]
[2]
[0]
[1]
[3]
[]
[0]
[1]
[]
[]
[]
[0]
[1]
[2]
[0]
[1, 0]
[3, 0]
[2]
[0]
[]
[]
[]
[0]
[1]
[3]
[2]
[0]
[]
[]
[2]
[]
[]
[3]
[1, 2, 0]
[0]
[]
[]
[]
[]
[]
[]
[2]
[]
[1]
[]
[2, 1]
[]
[]
[3]
[2]
[]
[0, 1]
[]
[]
[0]
[1]
[3]
[2]
[0]
[]
[]
[]
[0]
[]
[]
[2]
[]
[]
[]


In [11]:
predictions=seqmot2.predict(seqs, return_list=False)
for p in predictions: print p

1
1
0
3
1
1
1
1
0
1
1
0
0
0
1
1
0
1
0
1
1
1
0
1
0
1
0
1
0
0
1
3
1
1
1
1
0
1
1
0
0
0
1
1
1
1
2
2
1
1
0
0
0
1
1
1
1
1
0
0
1
0
0
1
3
1
0
0
0
0
0
0
1
0
1
0
2
0
0
1
1
0
2
0
0
1
1
1
1
1
0
0
0
1
0
0
1
0
0
0


In [12]:
predictions=seqmot2.transform(seqs, return_match=True)
for p in predictions: print p

[[], [], [(45, 55)], []]
[[(43, 52)], [], [], []]
[[], [], [], []]
[[], [(21, 30)], [(2, 11)], [(45, 54)]]
[[], [], [(43, 52)], []]
[[(26, 35), (45, 54)], [], [], []]
[[], [], [], [(19, 29)]]
[[], [], [], [(46, 55)]]
[[], [], [], []]
[[(45, 55)], [], [], []]
[[], [(45, 55)], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [(45, 55)], [], []]
[[], [(82, 92)], [], []]
[[], [], [], []]
[[(68, 77)], [], [], []]
[[], [], [], []]
[[], [], [], [(45, 54), (45, 55), (46, 55)]]
[[], [], [(44, 53), (45, 54), (45, 55), (46, 55), (47, 56)], []]
[[(42, 52)], [], [], []]
[[], [], [], []]
[[], [], [], [(42, 52)]]
[[], [], [], []]
[[(44, 54)], [], [], []]
[[], [], [], []]
[[], [], [], [(45, 55)]]
[[], [], [], []]
[[], [], [], []]
[[], [(43, 52)], [], []]
[[(33, 43)], [(14, 23)], [], [(46, 56)]]
[[], [], [(44, 54)], []]
[[(43, 53), (44, 54), (45, 55)], [], [], []]
[[], [(25, 34), (47, 56)], [], []]
[[], [], [], [(48, 58)]]
[[], [], [], []]
[[(45, 54)], [], [], []]
[[], [(45, 55), (46, 56

In [13]:
predictions=seqmot2.transform(seqs, return_match=False)
for p in predictions: print p

[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 1, 1]
[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[1, 1, 0, 1]
[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0]
[1, 0, 0, 0]
[1, 1, 0, 0]
[1, 0, 0, 1]
[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 1]
[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
[1, 1, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 1, 1, 0]

In [14]:
%%time
from sklearn.cluster import MiniBatchKMeans
ca = MiniBatchKMeans(n_clusters=4)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(training_size=100, clustering_algorithm=ca)
seqmot.fit(seqs)

for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

Positive data: Instances: 100 ; Features: 1048577 with an avg of 1831 features per instance
Negative data: Instances: 200 ; Features: 1048577 with an avg of 1842 features per instance
Elapsed time: 5.0 secs
model induction: 100 positive instances 5 s
motives extraction: 71 motives in 2s
motives clustering: 4 clusters in 1s
after filtering: 65 motives 4 clusters in 0s
motif model construction in 0s
updated motif counts in 0s
0
CCCCCCCCA 4
GCCCCCCCCA 3
CCCCCCCCCU 3
CCCCCCCCCC 3
UCCCCCCCG 2
UCCCCCCCCC 2
UCCCCCCCA 2
GCCCCCCCG 1
GCCCCCCCCU 1
CCCCCUCUCC 1
CCCCCGCCCC 1
CCCCCCACCU 1
ACUCCCCUCA 1
ACCCCCCCU 1
ACCCCCCCA 1
ACACCCACCC 1
1
UUUUUUUUU 4
UUUUUUUUUU 2
UUUUUUUUCU 2
UUAUUUUUA 2
CUUUUAUUU 2
AUUUUUUUUU 2
AUCUCUCUU 2
UUUUUCUGGU 1
UUUUAUUUUU 1
UUUGUUUUUU 1
UCUUUUUAUU 1
UCUCUUUUU 1
CUUUUUUCU 1
CUUUAUUUU 1
AUUUUUUUUA 1
AUUUUUUAUU 1
AGCUUCUUU 1
2
UAAAAAAAAA 3
AAAAAGAAAA 3
GAAAAAAGA 2
AAAAAAAGAA 2
AAAAAAAAAA 2
UAAUAAAGC 1
UAAAAAAGAG 1
UAAAAAAAAU 1
UAAAAAAAAG 1
GGAAAAAAAA 1
GAGAAAAAG 1
GAAAAAAAU 1

In [15]:
%%time
from sklearn.cluster import DBSCAN
ca = DBSCAN(eps=0.2, min_samples=3)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(training_size=100, clustering_algorithm=ca)
seqmot.fit(seqs)

for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

Positive data: Instances: 100 ; Features: 1048577 with an avg of 1831 features per instance
Negative data: Instances: 200 ; Features: 1048577 with an avg of 1842 features per instance
Elapsed time: 4.9 secs
model induction: 100 positive instances 5 s
motives extraction: 77 motives in 2s
motives clustering: 3 clusters in 0s
after filtering: 3 motives 2 clusters in 0s
motif model construction in 0s
updated motif counts in 0s
0
GCCCCCCCCA 3
1
CCCCCCCCC 5
CCCCCCCCCC 3
CPU times: user 3.5 s, sys: 1.3 s, total: 4.8 s
Wall time: 7.8 s
